# Seminar_1 

Exercise 1

In [1]:
class RGBto_YUV:
    def YUV_to_RGB(Y, U,V):
        B = round(1.163 * (Y -16) + 2.018 * (U - 128))
        G = round(1.164 * (Y - 16) - 0.813 * (V - 128) - 0.391 * (U - 128))
        R = round(1.164 * (Y - 16) + 1.596 * (V - 128)) 
        return(R, G, B)
    
    def RGB_to_YUV(R,G,B):
        Y = round(0.257 * R + 0.504 * G + 0.098 * B +16)
        U = round(-0.148 * R - 0.291 * G + 0.439 * B +128)
        V = round(0.439 * R - 0.368 * G - 0.071 * B +128)
        return(Y,U,V)
        

In [2]:
Y, U, V = 100, 128, 128

# Convertir YUV a RGB
R, G, B = RGBto_YUV.YUV_to_RGB(Y, U, V)
print(f"Valores RGB: R = {R}, G = {G}, B = {B}")

Valores RGB: R = 98, G = 98, B = 98


In [3]:
# Convertir RGB a YUV
Y, U, V = RGBto_YUV.RGB_to_YUV(R, G, B)
print(f"Valores YUV: Y = {Y}, U = {U}, V = {V}")

Valores YUV: Y = 100, U = 128, V = 128


Exercise 2

In [4]:
import subprocess

input_image = "/Users/viktoriaolmedo/Downloads/escut-barca.png"
output_image = "output_320x240.jpg"
#modificar per path de cada ordinador


def resize_image(input_image, output_image, width, height, quality = 28):

    command = ["ffmpeg","-y","-loglevel", "info","-i", input_image,"-vf", f"scale={width}:{height}",output_image ]

    try:
        result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print(result.stdout)
        print(f"Resized image saved to {output_image}")
    except subprocess.CalledProcessError as e:
        print("Error ocurred while running FFmpeg", e.stderr)


#ffmpeg -i input.jpg -vf scale=320:240 output_320x240.png

resize_image(input_image, output_image, 320, 240, quality=28)


Resized image saved to output_320x240.jpg


**Exercise 3**

We first implement it with a matrix 8x8 in order to check if the algorithm is really working. 

In [5]:
import numpy as np

matrix = np.arange(1, 65).reshape(8, 8)

def zigzag_traversal(matrix):
    result = [None] * 64  
    n = matrix.shape[0]  
    
    a = 0  
    i = 0 
    j = 0 
    result[a] = matrix[i][j]  # First value
    
    while a < 63:  
        #step 1: (one to the right)
        if j == n - 1:  
            i += 1
        else:
            j += 1

        a += 1
        if a < 64:
            result[a] = matrix[i][j]
        
        #step 2: (Diagonal down to the left until reaching j = 0)
        while i + 1 < n and j - 1 >= 0 and a < 63:
            i += 1
            j -= 1
            a += 1
            result[a] = matrix[i][j]
        
        #step 3: (one down) 
        if i == n - 1:  
            j += 1
        else:
            i += 1

        a += 1
        if a < 64:
            result[a] = matrix[i][j]
        
        #step 4: (Diagonal up to the right until reaching i = 0)
        while j + 1 < n and i - 1 >= 0 and a < 63:
            i -= 1
            j += 1
            a += 1
            result[a] = matrix[i][j]
    
    return result

# Obtener el recorrido zig-zag
zigzag_result = zigzag_traversal(matrix)

# Mostrar la matriz y el recorrido zig-zag
print("Matriz 8x8:")
print(matrix)
print("\nRecorrido Zig-Zag:")
print(zigzag_result)




Matriz 8x8:
[[ 1  2  3  4  5  6  7  8]
 [ 9 10 11 12 13 14 15 16]
 [17 18 19 20 21 22 23 24]
 [25 26 27 28 29 30 31 32]
 [33 34 35 36 37 38 39 40]
 [41 42 43 44 45 46 47 48]
 [49 50 51 52 53 54 55 56]
 [57 58 59 60 61 62 63 64]]

Recorrido Zig-Zag:
[1, 2, 9, 17, 10, 3, 4, 11, 18, 25, 33, 26, 19, 12, 5, 6, 13, 20, 27, 34, 41, 49, 42, 35, 28, 21, 14, 7, 8, 15, 22, 29, 36, 43, 50, 57, 58, 51, 44, 37, 30, 23, 16, 24, 31, 38, 45, 52, 59, 60, 53, 46, 39, 32, 40, 47, 54, 61, 62, 55, 48, 56, 63, 64]


Then we implemented for images. We use the resize_image function generated in the previous exercise in order to obtain a nxn image. Then we implement the zig zag function and obtain the expected output. First we implemented in Black and White, but then we realize that maybe it was needed to implemented for rgb images, so we added it to the code.

We can try and change the values of the resize image. In order to check if the algorithm is still working for this case with images, we have set the values to 16, because then we can see the complete matrix. 
Then we change it to a bigger value, 420 in this case.  

In [6]:
import numpy as np
import cv2
import subprocess

def load_image_bw(image_path):
    return cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  

def load_image_rgb(image_path):
    return cv2.imread(image_path)  

def serpentine(matrix):
    n = matrix.shape[0]  
    result = [None] * (n * n)  
    
    a = 0  
    i = 0 
    j = 0 
    result[a] = matrix[i, j]  # First value
    
    while a < n * n - 1:  
        #step 1: (one to the right)
        if j == n - 1:  
            i += 1
        else:
            j += 1

        a += 1
        if a < n * n:
            result[a] = matrix[i, j]
        
        #step 2: (Diagonal down to the left until reaching j = 0)
        while i + 1 < n and j - 1 >= 0 and a < n * n - 1:
            i += 1
            j -= 1
            a += 1
            result[a] = matrix[i, j]
        
        #step 3: (one down)
        if i == n - 1:  
            j += 1
        else:
            i += 1

        a += 1
        if a < n * n:
            result[a] = matrix[i, j]
        
        #step 4: (Diagonal up to the right until reaching i = 0)
        while j + 1 < n and i - 1 >= 0 and a < n * n - 1:
            i -= 1
            j += 1
            a += 1
            result[a] = matrix[i, j]
    
    return result


# Inputs and Outputs
image = "/Users/isall/OneDrive/UNI/4_uni/1_trim_4/Audio/video_coding/first_seminar/logo_fcb.jpeg"
input_image = "/Users/isall/OneDrive/UNI/4_uni/1_trim_4/Audio/video_coding/first_seminar/logo_fcb_resize.jpeg"
resize_image(image, input_image, 420, 420, quality=28)
output_image_bw = "/Users/isall/OneDrive/UNI/4_uni/1_trim_4/Audio/video_coding/first_seminar/serpentine_bw.jpeg"
output_image_rgb = "/Users/isall/OneDrive/UNI/4_uni/1_trim_4/Audio/video_coding/first_seminar/serpentine_rgb.jpeg"

##################### Black and White case #####################################

#We load the resize image in BW
image_bw = load_image_bw(input_image)

if image_bw is None:
    print("Error loading the BW image")
else:
    # Print the matrix to then check it the convertion is done well. 
    print("Original Matrix in Black and White:")
    print(image_bw)


# Check if it's a nxn matrix
if image_bw is None:
    print("Error loading the BW image")
else:
    assert image_bw.shape[0] == image_bw.shape[1], "The resize BW image is not nxn."

    serpentine_result_bw = serpentine(image_bw)

    print("Serpentine of image in Black and White:")
    print(serpentine_result_bw)

    n = image_bw.shape[0]
    serpentine_image_bw = np.array(serpentine_result_bw).reshape((n, n))

    # Safe the output
    cv2.imwrite(output_image_bw, serpentine_image_bw)
    print(f"Output in BW safe in: {output_image_bw}")


############################# RGB CASE ############################
def serpentine_color(image):
    channels = cv2.split(image)  # Separate into B, G, and R channels
    serpentine_channels = []

    for idx, channel in enumerate(channels):
        serpentine_result = serpentine(channel)
        
        # Print serpentine result for each channel
        print(f"Serpentine traversal output for channel {idx}:")
        print(serpentine_result)
        
        n = channel.shape[0]
        serpentine_matrix = np.array(serpentine_result).reshape((n, n))
        serpentine_channels.append(serpentine_matrix)

    # Merge channels back into a color image
    serpentine_image_color = cv2.merge(serpentine_channels)
    return serpentine_image_color


#We load the resize image in BW
image_rgb = load_image_rgb(input_image)


if image_rgb is None:
    print("Error loading the image in rgb")
else:
    # Print the matrix to then check it the conversion is done well. 
    print("Original Matrix rgb:")
    print(image_rgb)


if image_rgb is None:
    print("Error loading the image")
else:
    # Check image is nxn
    assert image_rgb.shape[0] == image_rgb.shape[1], "Input image rgb is not nxn."
    
    serpentine_result_rbg = serpentine_color(image_rgb)
    
    print("Output matrix of rgb image",serpentine_result_rbg)
    
    cv2.imwrite(output_image_rgb, serpentine_result_rbg)
    print(f"Serpentine image in rgb safe in: {output_image_rgb}")


ModuleNotFoundError: No module named 'cv2'

In [7]:
def compress_to_bw(input_image_path, output_image_path):
    try:
        subprocess.run([
            'ffmpeg', '-i', input_image_path, 
            '-vf', 'format=gray',  # Convert to black and white
            '-compression_level', '10',  # Max compression for PNG (range 0-10)
            '-qscale:v', '31',  # Highest compression for JPEG (range 2-31)
            '-y', output_image_path
        ], check=True)
        
        print(f"Compressed and converted image saved at: {output_image_path}")
        
    except subprocess.CalledProcessError as e:
        print(f"FFMPEG failed with error: {e}")

output_image_compressed = "/Users/isall/OneDrive/UNI/4_uni/1_trim_4/Audio/video_coding/first_seminar/output_image_compressed.jpeg"
compress_to_bw(output_image_rgb, output_image_compressed)

NameError: name 'output_image_rgb' is not defined

**Exercise 6:**

In [24]:
def encoding(byte_sequence):
    if not isinstance(byte_sequence, (bytes, bytearray)):
        raise ValueError("Input must be a bytes or bytearray object.")
    
    encoded_bytes = bytearray()
    i = 0
    
    while i < len(byte_sequence):
        count = 1
        current_byte = byte_sequence[i]
        j = i

        while j < len(byte_sequence) - 1:
            if byte_sequence[j] == byte_sequence[j + 1]:
                count += 1
                j += 1
            else:
                break

        encoded_bytes.append(count)
        encoded_bytes.append(current_byte)
        
        i = j + 1
    
    return bytes(encoded_bytes)

input_bytes = bytes([0, 65, 65, 65, 66, 66, 66, 67, 67, 68, 65, 65])
encoded_result = encoding(input_bytes)
print(list(encoded_result))  


[1, 0, 3, 65, 3, 66, 2, 67, 1, 68, 2, 65]


**Exercise 7**

In [25]:
import numpy as np
from scipy.fftpack import dct, idct

class DCTProcessor:
    def __init__(self):
        pass
    
    def apply_dct(self, block):
        if not isinstance(block, np.ndarray):
            raise ValueError("Input block must be a numpy array.")
        
        # Apply 2D DCT (type-II) along both axes
        return dct(dct(block.T, norm='ortho').T, norm='ortho')
    
    def apply_idct(self, dct_block):
        if not isinstance(dct_block, np.ndarray):
            raise ValueError("Input DCT block must be a numpy array.")
        
        return idct(idct(dct_block.T, norm='ortho').T, norm='ortho')

# Example:
if __name__ == "__main__":
    dct_processor = DCTProcessor()

    block = np.array([
        [52, 55, 61, 66, 70, 61, 64, 73],
        [63, 59, 66, 90, 109, 85, 69, 72],
        [62, 59, 68, 113, 144, 104, 66, 73],
        [63, 58, 71, 122, 154, 106, 70, 69],
        [67, 61, 68, 104, 126, 88, 68, 70],
        [79, 65, 60, 70, 77, 68, 58, 75],
        [85, 71, 64, 59, 55, 61, 65, 83],
        [87, 79, 69, 68, 65, 76, 78, 94]
    ])

    # Apply DCT
    dct_block = dct_processor.apply_dct(block)
    print("DCT Result:\n", np.round(dct_block, 2))

    # Apply IDCT to reconstruct
    reconstructed_block = dct_processor.apply_idct(dct_block)
    print("\nReconstructed Block:\n", np.round(reconstructed_block, 2))


DCT Result:
 [[ 6.100e+02 -2.911e+01 -6.194e+01  2.533e+01  5.475e+01 -1.972e+01
  -5.900e-01  2.080e+00]
 [ 6.080e+00 -2.059e+01 -6.163e+01  8.010e+00  1.153e+01 -6.640e+00
  -6.420e+00  6.780e+00]
 [-4.609e+01  7.960e+00  7.673e+01 -2.559e+01 -2.966e+01  1.014e+01
   6.390e+00 -4.770e+00]
 [-4.891e+01  1.177e+01  3.431e+01 -1.423e+01 -9.860e+00  6.190e+00
   1.340e+00  1.500e+00]
 [ 1.075e+01 -7.630e+00 -1.245e+01 -2.040e+00 -5.000e-01  1.370e+00
  -4.580e+00  1.520e+00]
 [-9.640e+00  1.410e+00  3.410e+00 -3.290e+00 -4.700e-01  4.200e-01
   1.810e+00 -3.900e-01]
 [-2.830e+00 -1.230e+00  1.390e+00  8.000e-02  9.200e-01 -3.510e+00
   1.770e+00 -2.770e+00]
 [-1.250e+00 -7.100e-01 -4.900e-01 -2.690e+00 -9.000e-02 -4.000e-01
  -9.100e-01  4.100e-01]]

Reconstructed Block:
 [[ 52.  55.  61.  66.  70.  61.  64.  73.]
 [ 63.  59.  66.  90. 109.  85.  69.  72.]
 [ 62.  59.  68. 113. 144. 104.  66.  73.]
 [ 63.  58.  71. 122. 154. 106.  70.  69.]
 [ 67.  61.  68. 104. 126.  88.  68.  70.]
 [ 7

In [27]:
from scipy import fft 

block = np.array([
    [52, 55, 61, 66, 70, 61, 64, 73],
    [63, 59, 66, 90, 109, 85, 69, 72],
    [62, 59, 68, 113, 144, 104, 66, 73],
    [63, 58, 71, 122, 154, 106, 70, 69],
    [67, 61, 68, 104, 126, 88, 68, 70],
    [79, 65, 60, 70, 77, 68, 58, 75],
    [85, 71, 64, 59, 55, 61, 65, 83],
    [87, 79, 69, 68, 65, 76, 78, 94]
])

gfg = fft.dct(block) 

print(gfg)

[[1004.          -57.72015737  -22.62145031  -26.96553606   28.28427125
   -12.33157164   -2.87575832    9.65275291]
 [1226.          -62.80862838 -135.86001805   47.31686293   77.78174593
   -29.39383609   -6.48502085   13.27377425]
 [1378.          -85.31450748 -261.39884858   89.50082767  134.35028843
   -64.08917035   -6.53008144    4.42887148]
 [1426.          -83.10039098 -303.58028174   98.91599692  145.66399692
   -49.9983738   -19.67263434   15.55998681]
 [1304.          -48.33206974 -192.50649839   61.4189483   115.96551211
   -33.99070329  -21.28962366   16.50320918]
 [1104.            7.86646858    9.10747913   27.39104014   70.71067812
   -24.04845179   14.59636338   -5.78978056]
 [1086.           18.79492044  108.19802502   -9.34447896   29.69848481
    -1.7248436    21.00446098   -8.74394608]
 [1232.          -18.67549603   97.8768375    -1.63318269   16.97056275
    -7.48200066   14.56450073  -21.36769123]]


In [35]:
from scipy.fftpack import dct

dct(block, 2)

#type=1 (DCT-I): This type is less commonly used. It has specific boundary conditions where the input and output are symmetric, meaning that the first and last elements are half-weighted.
#type=2 (DCT-II): This is the most commonly used type, especially in image compression (e.g., JPEG). It transforms a real input array into a real output array, with no symmetry in the boundary conditions.
#type=3 (DCT-III): This type is the inverse of DCT-II.
#type=4 (DCT-IV): Less common and typically used in specialized applications.


array([[1004.        ,  -57.72015737,  -22.62145031,  -26.96553606,
          28.28427125,  -12.33157164,   -2.87575832,    9.65275291],
       [1226.        ,  -62.80862838, -135.86001805,   47.31686293,
          77.78174593,  -29.39383609,   -6.48502085,   13.27377425],
       [1378.        ,  -85.31450748, -261.39884858,   89.50082767,
         134.35028843,  -64.08917035,   -6.53008144,    4.42887148],
       [1426.        ,  -83.10039098, -303.58028174,   98.91599692,
         145.66399692,  -49.9983738 ,  -19.67263434,   15.55998681],
       [1304.        ,  -48.33206974, -192.50649839,   61.4189483 ,
         115.96551211,  -33.99070329,  -21.28962366,   16.50320918],
       [1104.        ,    7.86646858,    9.10747913,   27.39104014,
          70.71067812,  -24.04845179,   14.59636338,   -5.78978056],
       [1086.        ,   18.79492044,  108.19802502,   -9.34447896,
          29.69848481,   -1.7248436 ,   21.00446098,   -8.74394608],
       [1232.        ,  -18.67549603,   9